# Run & display stats

In [13]:
import gc
import os
import datetime
import numpy as np
from tqdm.notebook import tqdm

from sim import FunctionType, Simulation, StaticSimulation, SimulationResults, LinearSimulation, LinearSimulationStatic, LongstepSimulation, LongstepSimulationStatic, SimulationEnsemble
from graph import GraphType, CompleteGraph, RingGraph, RocGraph, ErGraph, CompleteGraph
from rw_utils import pickle_obj
from db import SimulationDatabase, SimulationDirectory

In [14]:
num_nodes = 100
nums_edges = [220, 460, 1204]
graph_types = [GraphType.ring, GraphType.roc, GraphType.er]

# (GraphType, num_edges): Graph
graphs = {
    # (GraphType.complete, 4950): (CompleteGraph, {'num_nodes': num_nodes}),
    (GraphType.ring, 220): (RingGraph, {'num_nodes': num_nodes, 'degree': 4}),
    (GraphType.ring, 460): (RingGraph, {'num_nodes': num_nodes, 'degree': 10}),
    (GraphType.ring, 1204): (RingGraph, {'num_nodes': num_nodes, 'degree': 24}),
    (GraphType.er, 220): (ErGraph, {'num_nodes': num_nodes, 'num_edges': 220}),
    (GraphType.er, 460): (ErGraph, {'num_nodes': num_nodes, 'num_edges': 460}),
    (GraphType.er, 1204): (ErGraph, {'num_nodes': num_nodes, 'num_edges': 1204}),
    (GraphType.roc, 220): (RocGraph, {'num_cliques': 20, 'clique_size': 5}),
    (GraphType.roc, 460): (RocGraph, {'num_cliques': 10, 'clique_size': 10}),
    (GraphType.roc, 1204): (RocGraph, {'num_cliques': 4, 'clique_size': 25})
}

# (FunctionType, static): Simulation
simulations = {
    (FunctionType.linear, False): LinearSimulation,
    (FunctionType.longstep, False): LongstepSimulation,
    (FunctionType.linear, True): LinearSimulationStatic,
    (FunctionType.longstep, True): LongstepSimulationStatic
}

In [15]:
num_runs = 5
params = {
    'graph': None,
    'eps': 0.1,
    'beta_0': 0.055,
    'beta_1': 0.050,
    'alpha_0': 0.0,
    'alpha_1': 0.0,
    'X': np.array([[.02, .98], [.02, .98]]),
    'max_num_iter': 150_000
}

## Run

Careful with parameters for `Longstep, RocGraph(4x25)` and `Longstep, RingGraph(100,24)`.

In [ ]:
for (graph_type, num_edges), (graph_class, graph_params) in graphs.items():
    for (func_type, static), simulation_class in simulations.items():
        params['graph'] = graph_class(**graph_params)
        sim = simulation_class(**params)
        sim_ensemble = SimulationEnsemble(sim, num_runs)
        sim_ensemble.run()

## Display stats

In [ ]:
sdb = SimulationDatabase(path='output')
for (graph_type, num_edges), (_, graph_params) in graphs.items():
    for (func_type, static), _ in simulations.items():
        sdirs = [
            sdir
            for sdir in sdb.sdirs
            if (sdir.sim.func_type == func_type) and
            (static is isinstance(sdir.sim, StaticSimulation)) and
            (num_edges - 70 <= sdir.sim.graph.graph.number_of_edges() <= num_edges + 70) and
            (sdir.sim.graph.graph_type == graph_type)
        ]
        if len(sdirs) != 1:
            raise RuntimeError(f"Found {len(sdirs)} SimulationDirectories after filter. Expected 1.")
        sdir = sdirs[0]
        sdir.cache()
        sdir.info()
        del sdir
        gc.collect()

# Plot & animate

In [18]:
import os
import gc
import numpy as np
from matplotlib import pyplot as plt

from sim import FunctionType, StaticSimulation, FunctionType
from graph import GraphType, RingGraph, RocGraph, ErGraph
from plot import SigmaPlot
from db import SimulationDatabase, SimulationDirectory
from utils import sigma_mean, sigmas, pad
from rw_utils import read_pickled, deserialize_boolean_array
from animate import animate

%matplotlib inline

In [19]:
color = {
    GraphType.ring: 'red',
    GraphType.roc: 'green',
    GraphType.er: 'blue',
}

x_right = {
    FunctionType.linear: 2500,
    FunctionType.longstep: 25000
}

label = {
    GraphType.ring: 'k-рег. кольцо',
    GraphType.roc: 'кольцо клик',
    GraphType.er: 'ER',
}

## Plot all

In [ ]:
statics = [
    False,
    True
]

func_types = [
    FunctionType.linear,
    FunctionType.longstep
]

nums_edges = [
    220,
    460,
    1204
]

sdb = SimulationDatabase(path='output')
for static in statics:
    for func_type in func_types:
        for num_edges in nums_edges:
            
            sdirs = [
                sdir
                for sdir in sdb.sdirs
                if (sdir.sim.func_type == func_type) and
                (static is isinstance(sdir.sim, StaticSimulation)) and
                (num_edges - 70 <= sdir.sim.graph.graph.number_of_edges() <= num_edges + 70) and
                (sdir.sim.graph.graph_type != GraphType.complete)
            ]
            if len(sdirs) != 3:
                raise RuntimeError(f"Expected 3 SimulationDirectories after filter, got {len(sdirs)}.")
            
            sim = sdirs[1].sim
            fname = f"{sim.func_type}_e{sim.graph.graph.number_of_edges()}" + "_static" * static + ".png"
            print(f"{fname}:")
            splot = SigmaPlot(linewidth=3.5, alpha=.8, ax_tick_size=18, ax_label_size=28)
            
            gc.collect()
            
            for sdir in sdirs:
                print(f'caching {sdir}...')
                sdir.cache()
            print()
            x_max = max([sdir.t_max for sdir in sdirs])
            
            for sdir in sdirs:
                splot.plot(sdir, color=color[sdir.sim.graph.graph_type], x_max=x_max, label=label[sdir.sim.graph.graph_type])
            xright = 2500 if static else x_right[sim.func_type]
            splot.update_axlims(x_right=xright)

            # display
            display(splot.fig)

            # # save
            # splot.fig.savefig(
            #     os.path.join('plots', fname),
            #     bbox_inches='tight',
            #     pad_inches=0
            # )

## Plot three

In [ ]:
num_edges = 1204
sdb = SimulationDatabase(path='output')
sdirs = [
    sdir
    for sdir in sdb.sdirs
    if sdir.sim.func_type == FunctionType.linear and
    not isinstance(sdir.sim, StaticSimulation) and
    num_edges - 70 <= sdir.sim.graph.graph.number_of_edges() <= num_edges + 70 and
    sdir.sim.graph.graph_type != GraphType.complete
]
print(sdirs)
for sdir in sdirs:
    sdir.cache()

In [ ]:
%%capture
x_max = max([sdir.t_max for sdir in sdirs])
splot = SigmaPlot(linewidth=3.5, alpha=.8, ax_tick_size=18, ax_label_size=28)
# splot.x_label = "время"
# splot.y_label = "доля вершин в состоянии 1"
# splot._setup()
for sdir in sdirs:
    splot.plot(sdir, color=color[sdir.sim.graph.graph_type], x_max=x_max)
splot.update_axlims(x_right=2500)

### Display

In [ ]:
display(splot.fig)

### Save

In [8]:
splot.fig.savefig(
    os.path.join('plots', fname),
    bbox_inches='tight',
    pad_inches=0
)

## Animate

In [ ]:
sdir = SimulationDirectory("output\\2023.12.06_19.43.01_longstep_er(n=100,m=1204)_static")
sr = read_pickled('4', path=sdir.path)
animate(
    sdir.sim,
    sr,
    filename='test',
    path=sdir.path,
    t_start=max(0, sr.t - 1000),
    t_end=sr.t,
    overwrite=True
)